## Database Functions

In [1]:
from kitab.db.functions import add_book_db, get_book_by_ISBN, add_book_db, update_book_db, get_table_from_db

c:\Users\alexa\Desktop\Spring 2024\DS 223 Marketing Analytics\Project\ds223-book-recommendation\book-recommendation\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
add_book_db(
    {"ISBN": "test_isbn",
     "title": "test_title",
     "description": "test_description",
     "available": True,
     "authors": ["test_author", "Agatha Christie"], 
     "genres": ["test_genre", "Fiction"]}
)

2024-04-09 22:36:38,008 - kitab.db.sql_interactions - insert_records - INFO - (1 records inserted successfully.) - line: 148
c:\Users\alexa\Desktop\Spring 2024\DS 223 Marketing Analytics\Project\ds223-book-recommendation\kitab\db\sql_interactions.py:241: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(query, self.connection)
2024-04-09 22:36:38,231 - kitab.db.sql_interactions - insert_records - WARNING - (No records to insert.) - line: 137
2024-04-09 22:36:38,272 - kitab.db.sql_interactions - insert_records - INFO - (2 records inserted successfully.) - line: 148
c:\Users\alexa\Desktop\Spring 2024\DS 223 Marketing Analytics\Project\ds223-book-recommendation\kitab\db\sql_interactions.py:241: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connectio

In [3]:
get_book_by_ISBN("test_isbn1")

c:\Users\alexa\Desktop\Spring 2024\DS 223 Marketing Analytics\Project\ds223-book-recommendation\kitab\db\sql_interactions.py:238: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(query, self.connection, params=values)


In [4]:
get_table_from_db("bookauthor").iloc[-10:]

c:\Users\alexa\Desktop\Spring 2024\DS 223 Marketing Analytics\Project\ds223-book-recommendation\kitab\db\sql_interactions.py:241: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(query, self.connection)


,isbn,author_id
123901,some_isabn1,74396
123902,some_isasdabn1,74396
123903,some_isasdasasbn1,74396
123904,some_isasdasbn1,74397
123905,teasast_isbn1,74395
123906,teasast_isbn1,1676
123907,test_isbsan2,1676
123908,test_isbsan2,74394
123909,test_isbn,74395
123910,test_isbn,1676


In [5]:
update_book_db("test_isbn1", {"ISBN": "test_isbsan2", "authors": ["Agatha Christie", "test_author2"]})

2024-04-09 22:36:38,932 - kitab.db.sql_interactions - update_records - INFO - (Records updated successfully.) - line: 179
c:\Users\alexa\Desktop\Spring 2024\DS 223 Marketing Analytics\Project\ds223-book-recommendation\kitab\db\sql_interactions.py:238: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(query, self.connection, params=values)
c:\Users\alexa\Desktop\Spring 2024\DS 223 Marketing Analytics\Project\ds223-book-recommendation\kitab\db\sql_interactions.py:241: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(query, self.connection)
2024-04-09 22:36:39,133 - kitab.db.sql_interactions - insert_records - WARNING - (No records to insert.) - line

In [6]:
get_book_by_ISBN("test_isbn1")

c:\Users\alexa\Desktop\Spring 2024\DS 223 Marketing Analytics\Project\ds223-book-recommendation\kitab\db\sql_interactions.py:238: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(query, self.connection, params=values)


In [7]:
get_book_by_ISBN("test_isbn2")

c:\Users\alexa\Desktop\Spring 2024\DS 223 Marketing Analytics\Project\ds223-book-recommendation\kitab\db\sql_interactions.py:238: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(query, self.connection, params=values)
c:\Users\alexa\Desktop\Spring 2024\DS 223 Marketing Analytics\Project\ds223-book-recommendation\kitab\db\sql_interactions.py:238: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(query, self.connection, params=values)


{'isbn': 'test_isbn2',
 'title': 'test_title',
 'description': 'test_description',
 'available': True,
 'authors': ['Agatha Christie', 'test_author2'],
 'genres': ['Fiction', 'test_genre']}

In [8]:
get_table_from_db("bookauthor").iloc[-10:]

c:\Users\alexa\Desktop\Spring 2024\DS 223 Marketing Analytics\Project\ds223-book-recommendation\kitab\db\sql_interactions.py:241: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(query, self.connection)


,isbn,author_id
123901,some_isabn1,74396
123902,some_isasdabn1,74396
123903,some_isasdasasbn1,74396
123904,some_isasdasbn1,74397
123905,teasast_isbn1,74395
123906,teasast_isbn1,1676
123907,test_isbsan2,1676
123908,test_isbsan2,74394
123909,test_isbn,74395
123910,test_isbn,1676


## Model

In [9]:
from kitab.recommendation_model.models import recommend_books, recommend_books_by_ISBN, recommend_books_by_title

In [10]:
sample_description = "The book projects American society immediately after World War II into a technologically advanced future where the amplification of humanity's potentials to create and destroy have both miraculous and devastating consequences."

In [11]:
recommended_books1 = recommend_books(sample_description, 5)

c:\Users\alexa\Desktop\Spring 2024\DS 223 Marketing Analytics\Project\ds223-book-recommendation\kitab\db\sql_interactions.py:241: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(query, self.connection)
c:\Users\alexa\Desktop\Spring 2024\DS 223 Marketing Analytics\Project\ds223-book-recommendation\kitab\recommendation_model\models.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  most_similar_books.drop(columns=["embedding"], inplace=True)
c:\Users\alexa\Desktop\Spring 2024\DS 223 Marketing Analytics\Project\ds223-book-recommendation\kitab\db\sql_interactions.py:238: UserWarning: pandas only supports SQLAlchemy connectable (engin

In [12]:
recommended_books1

[{'isbn': '345430247',
  'title': 'Aftershocks',
  'description': '\n  ,Set in the same universe as the ,Worldwar, series, ,Colonization: Aftershocks, brings us to Earth in the 1960s, where four superpowers rule: the United States, Russia, Nazi Germany, and the aliens of the Race.,\n,The arrival of the aliens shot Earth technology forward at a dizzying rateâ€”the world of 1960 includes routine space shuttle traffic, computers on every desk, and other high tech advancements. Aftershocks reveals that it was the United States that launched the nuclear offensive against the aliens, and when the aliens learn the truth, Indianapolis disappears in nuclear fire. In alien-held China, a full-fledged rebellion is launched, and in Poland, a nuclear bomb goes missingâ€”held by Jewish forces who seek to use it against the Nazis.,The nations of Earth begin to realize that their only hope in holding back the aliens will be to work together. Because separately, they may face extinction.,About the Autho

In [13]:
sample_isbn = "087584717X"

In [14]:
recommended_books2 = recommend_books_by_ISBN(sample_isbn, 5)

c:\Users\alexa\Desktop\Spring 2024\DS 223 Marketing Analytics\Project\ds223-book-recommendation\kitab\db\sql_interactions.py:241: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(query, self.connection)
c:\Users\alexa\Desktop\Spring 2024\DS 223 Marketing Analytics\Project\ds223-book-recommendation\kitab\recommendation_model\models.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  most_similar_books.drop(columns=["embedding"], inplace=True)
c:\Users\alexa\Desktop\Spring 2024\DS 223 Marketing Analytics\Project\ds223-book-recommendation\kitab\db\sql_interactions.py:238: UserWarning: pandas only supports SQLAlchemy connectable (engin

In [15]:
recommended_books2

[{'isbn': '087584717X',
  'title': 'Competitive Advantage Through People: Unleashing the Power of the Work Force',
  'description': '"Competitive Advantage Through People" explores why - despite long-standing evidence that a committed work force is essential for success - firms continue to attach little importance to their workers. The answer, argues Pfeffer, resides in a complex web of factors based on perception, history, legislation, and practice that continues to dominate management thought and action. Yet, some organizations have been able to overcome these obstacles. In fact, the five common stocks with the highest returns between 1972 and 1992 - Southwest Airlines, Wal-Mart, Tyson Foods, Circuit City, and Plenum Publishing - were in industries that shared virtually none of the characteristics traditionally associated with strategic success. What each of these firms did share is the ability to produce sustainable competitive advantage through its way of managing people. Pfeffer d

In [16]:
sample_title = "Spirit Wolf"

In [17]:
recommended_books3 = recommend_books_by_title(sample_title, 5)

c:\Users\alexa\Desktop\Spring 2024\DS 223 Marketing Analytics\Project\ds223-book-recommendation\kitab\db\sql_interactions.py:241: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(query, self.connection)
c:\Users\alexa\Desktop\Spring 2024\DS 223 Marketing Analytics\Project\ds223-book-recommendation\kitab\recommendation_model\models.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  most_similar_books.drop(columns=["embedding"], inplace=True)
c:\Users\alexa\Desktop\Spring 2024\DS 223 Marketing Analytics\Project\ds223-book-recommendation\kitab\db\sql_interactions.py:238: UserWarning: pandas only supports SQLAlchemy connectable (engin

In [18]:
recommended_books3

[{'isbn': '545279615',
  'title': 'Spirit Wolf',
  'description': "Faolan, a wolf once doomed to die, struggles to fufill his destiny as the leader of the wolves of Beyond.,No one ever saw Faolan as a leader. Banished as a pup, he survived and returned as a gnaw wolf-the lowest member of the wolf pack. But Faolan wasn't meant to be a gnaw wolf. It's not just his strange, splayed paw, or his uncanny connection with the bears. There's something about Faolan that inspires certain wolves . . . and leaves others deeply suspicious. Now, with a dangerous threat on the horizon, the pack must make a choice. Will they trust the silver outsider with the task of leading? If Faolan can't fulfill his destiny, it could be the end of the wolves of the Beyond.",
  'available': True,
  'authors': ['Kathryn Lasky'],
  'genres': ['Fiction',
   'Adventure',
   'Fantasy',
   'Magic',
   'Animals',
   'Childrens',
   'Middle Grade',
   'Young Adult',
   'Animal Fiction',
   'Wolves']},
 {'isbn': '545093163',